In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
import joblib

In [ ]:
model = joblib.load('xgboost_model.pkl')  # Replace with the correct model path

In [ ]:
scaler = RobustScaler()  # Re-initialize RobustScaler

In [ ]:
training_data = pd.read_csv('/content/creditcard.csv')
X_train = training_data.drop(['Class'], axis=1)  # Drop target column
scaler.fit(X_train)  # Fit scaler on training features

RobustScaler()

In [ ]:
num_synthetic_samples = 1000  # Number of synthetic transactions

In [ ]:
# Step 2: Generate synthetic data
np.random.seed(42)  # For reproducibility
synthetic_data = {
    "Time": np.random.randint(0, 172792, size=num_synthetic_samples),  # Example: time range in seconds
    "Amount": np.random.uniform(0.0, 5000.0, size=num_synthetic_samples),  # Amount range
}


In [ ]:
for i in range(1, 29):  # Features V1 to V28
    synthetic_data[f"V{i}"] = np.random.normal(0, 1, size=num_synthetic_samples)  # Mean=0, Std=1

In [ ]:
synthetic_transactions = pd.DataFrame(synthetic_data)

In [ ]:
# Ensure column order matches the training data
synthetic_transactions = synthetic_transactions[X_train.columns]


In [ ]:
# Step 3: Scale the synthetic data using RobustScaler
synthetic_transactions_scaled = scaler.transform(synthetic_transactions)

In [ ]:

# Step 4: Predict fraud probabilities using the trained model
predicted_probabilities = model.predict_proba(synthetic_transactions_scaled)[:, 1]  # Probability of fraud
predicted_classes = model.predict(synthetic_transactions_scaled)

In [ ]:
# Step 5: Add predictions to the synthetic dataset
synthetic_transactions['Fraud_Probability'] = predicted_probabilities
synthetic_transactions['Prediction'] = predicted_classes

In [ ]:
# Step 6: Display some results
print(synthetic_transactions.head())


     Time        V1        V2        V3        V4        V5        V6  \
0  121958  1.749242  0.285690 -0.230932 -0.238271 -0.864070 -0.592754   
1  146867  0.794017 -1.066721  0.684843 -1.241042 -0.240662  1.369329   
2  131932  0.457537 -0.116630 -0.965233  1.458639 -1.216607 -0.079389   
3  103694 -1.193597 -0.723088 -0.611269 -0.181636  0.982033  0.509139   
4  119879  0.935238 -0.401059 -1.167727  1.657142  1.717331  0.501561   

         V7        V8        V9  ...       V22       V23       V24       V25  \
0 -0.284905 -0.142413  0.471011  ...  0.474793 -0.931428 -2.305303 -0.022552   
1 -0.567262 -0.708133 -0.880185  ... -0.573194 -1.632434  1.168117  0.953462   
2  0.128481 -1.661279  0.226164  ... -0.399728 -1.055647 -0.672641  1.450519   
3 -0.592644 -0.260669  0.225804  ... -0.643047  1.517267  1.600411 -0.236680   
4  0.254206 -0.323815 -0.624454  ...  0.676223  0.331462  0.120488 -0.455019   

        V26       V27       V28       Amount  Fraud_Probability  Prediction  
0 

In [ ]:
synthetic_transactions.to_csv('synthetic_transactions_with_predictions.csv', index=False)

In [ ]:
print("Synthetic data with predictions saved to 'synthetic_transactions_with_predictions.csv'")

Synthetic data with predictions saved to 'synthetic_transactions_with_predictions.csv'
